<a href="https://colab.research.google.com/github/tiwaripari/ML/blob/main/SVM_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/MyDrive/Acoustic Features.csv")

In [4]:
df

,Class,_RMSenergy_Mean,_Lowenergy_Mean,_Fluctuation_Mean,_Tempo_Mean,_MFCC_Mean_1,_MFCC_Mean_2,_MFCC_Mean_3,_MFCC_Mean_4,_MFCC_Mean_5,...,_Chromagram_Mean_9,_Chromagram_Mean_10,_Chromagram_Mean_11,_Chromagram_Mean_12,_HarmonicChangeDetectionFunction_Mean,_HarmonicChangeDetectionFunction_Std,_HarmonicChangeDetectionFunction_Slope,_HarmonicChangeDetectionFunction_PeriodFreq,_HarmonicChangeDetectionFunction_PeriodAmp,_HarmonicChangeDetectionFunction_PeriodEntropy
0,relax,0.052,0.591,9.136,130.043,3.997,0.363,0.887,0.078,0.221,...,0.426,1.000,0.008,0.101,0.316,0.261,0.018,1.035,0.593,0.970
1,relax,0.125,0.439,6.680,142.240,4.058,0.516,0.785,0.397,0.556,...,0.002,1.000,0.000,0.984,0.285,0.211,-0.082,3.364,0.702,0.967
2,relax,0.046,0.639,10.578,188.154,2.775,0.903,0.502,0.329,0.287,...,0.184,0.746,0.016,1.000,0.413,0.299,0.134,1.682,0.692,0.963
3,relax,0.135,0.603,10.442,65.991,2.841,1.552,0.612,0.351,0.011,...,0.038,1.000,0.161,0.757,0.422,0.265,0.042,0.354,0.743,0.968
4,relax,0.066,0.591,9.769,88.890,3.217,0.228,0.814,0.096,0.434,...,0.004,0.404,1.000,0.001,0.345,0.261,0.089,0.748,0.674,0.957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,angry,0.174,0.578,5.265,161.144,2.985,0.065,0.718,0.046,0.265,...,0.374,0.838,0.413,0.125,0.323,0.140,0.120,2.691,0.867,0.969
396,angry,0.176,0.498,5.042,179.787,2.790,-0.148,0.342,-0.011,0.029,...,0.020,1.000,0.640,0.010,0.233,0.127,0.145,3.364,0.879,0.967
397,angry,0.187,0.557,4.724,134.032,1.672,0.566,0.880,-0.113,0.244,...,0.052,0.184,0.287,0.101,0.269,0.157,0.111,3.364,0.819,0.962
398,angry,0.140,0.573,4.470,113.600,1.997,-0.210,0.694,0.089,0.206,...,0.137,1.000,0.304,0.140,0.277,0.144,0.061,3.364,0.811,0.969


In [5]:
df.Class.value_counts()

relax    100
happy    100
sad      100
angry    100
Name: Class, dtype: int64

In [42]:
features = df.columns[1:]
features

Index(['_RMSenergy_Mean', '_Lowenergy_Mean', '_Fluctuation_Mean',
       '_Tempo_Mean', '_MFCC_Mean_1', '_MFCC_Mean_2', '_MFCC_Mean_3',
       '_MFCC_Mean_4', '_MFCC_Mean_5', '_MFCC_Mean_6', '_MFCC_Mean_7',
       '_MFCC_Mean_8', '_MFCC_Mean_9', '_MFCC_Mean_10', '_MFCC_Mean_11',
       '_MFCC_Mean_12', '_MFCC_Mean_13', '_Roughness_Mean', '_Roughness_Slope',
       '_Zero-crossingrate_Mean', '_AttackTime_Mean', '_AttackTime_Slope',
       '_Rolloff_Mean', '_Eventdensity_Mean', '_Pulseclarity_Mean',
       '_Brightness_Mean', '_Spectralcentroid_Mean', '_Spectralspread_Mean',
       '_Spectralskewness_Mean', '_Spectralkurtosis_Mean',
       '_Spectralflatness_Mean', '_EntropyofSpectrum_Mean',
       '_Chromagram_Mean_1', '_Chromagram_Mean_2', '_Chromagram_Mean_3',
       '_Chromagram_Mean_4', '_Chromagram_Mean_5', '_Chromagram_Mean_6',
       '_Chromagram_Mean_7', '_Chromagram_Mean_8', '_Chromagram_Mean_9',
       '_Chromagram_Mean_10', '_Chromagram_Mean_11', '_Chromagram_Mean_12',
      

In [43]:
#Creating Test and Train data
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75
#View top 10 rows
df.head(10)

,Class,_RMSenergy_Mean,_Lowenergy_Mean,_Fluctuation_Mean,_Tempo_Mean,_MFCC_Mean_1,_MFCC_Mean_2,_MFCC_Mean_3,_MFCC_Mean_4,_MFCC_Mean_5,...,_Chromagram_Mean_10,_Chromagram_Mean_11,_Chromagram_Mean_12,_HarmonicChangeDetectionFunction_Mean,_HarmonicChangeDetectionFunction_Std,_HarmonicChangeDetectionFunction_Slope,_HarmonicChangeDetectionFunction_PeriodFreq,_HarmonicChangeDetectionFunction_PeriodAmp,_HarmonicChangeDetectionFunction_PeriodEntropy,is_train
0,relax,0.052,0.591,9.136,130.043,3.997,0.363,0.887,0.078,0.221,...,1.000,0.008,0.101,0.316,0.261,0.018,1.035,0.593,0.970,True
1,relax,0.125,0.439,6.680,142.240,4.058,0.516,0.785,0.397,0.556,...,1.000,0.000,0.984,0.285,0.211,-0.082,3.364,0.702,0.967,True
2,relax,0.046,0.639,10.578,188.154,2.775,0.903,0.502,0.329,0.287,...,0.746,0.016,1.000,0.413,0.299,0.134,1.682,0.692,0.963,True
3,relax,0.135,0.603,10.442,65.991,2.841,1.552,0.612,0.351,0.011,...,1.000,0.161,0.757,0.422,0.265,0.042,0.354,0.743,0.968,True
4,relax,0.066,0.591,9.769,88.890,3.217,0.228,0.814,0.096,0.434,...,0.404,1.000,0.001,0.345,0.261,0.089,0.748,0.674,0.957,True
5,relax,0.137,0.510,7.664,116.044,2.313,-0.082,0.365,0.033,-0.136,...,0.391,0.102,0.000,0.280,0.240,0.355,1.682,0.662,0.958,True
6,relax,0.123,0.533,7.483,136.444,4.572,1.171,0.939,0.358,0.024,...,1.000,0.000,0.143,0.210,0.174,0.193,2.691,0.691,0.966,True
7,relax,0.103,0.575,7.006,141.132,3.495,0.553,0.275,0.269,0.257,...,1.000,0.030,0.746,0.366,0.267,-0.061,0.585,0.711,0.965,False
8,relax,0.145,0.542,6.846,82.048,4.010,0.550,0.568,0.559,0.218,...,0.150,0.910,0.000,0.322,0.210,0.031,0.408,0.796,0.970,True
9,relax,0.129,0.618,8.668,93.020,2.318,0.744,0.537,-0.116,0.119,...,0.018,0.385,0.000,0.388,0.340,0.122,1.682,0.627,0.965,False


In [44]:
#Create dataframes with train and test rows
train, test = df[df['is_train']==True], df[df['is_train']==False]

#show the number of rows in test and train dataframe
print("training data ", len(train))
print("testing data ", len(test))

training data  292
testing data  108


In [45]:
#Converting each Class naem into digits
y = pd.factorize(train['Class'])[0]
#viewing target
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3])

In [46]:
#Converting each Class name into digits for test data
x = pd.factorize(test['Class'])[0]
#viewing target
x

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [47]:
target_names = (train['Class']).unique()
target_names


array(['relax', 'happy', 'sad', 'angry'], dtype=object)

In [48]:
# Create an SVM classifier
svm_classifier = SVC(kernel='linear')

# Train the classifier
svm_classifier.fit(train[features], y)

# Make predictions on the test set
y_pred = svm_classifier.predict(test[features])

In [49]:
#mapping names of moods for each music for each predicted mood
preds  = target_names[svm_classifier.predict(test[features])]

#View the predicted moods for firat five observations
preds[10:20]

array(['relax', 'relax', 'relax', 'sad', 'relax', 'relax', 'relax',
       'relax', 'relax', 'sad'], dtype=object)

In [50]:
pd.crosstab(test['Class'], preds, rownames=['Actual mood'], colnames=['Predicted Species'])

Predicted Species,angry,happy,relax,sad
Actual mood,,,,
angry,15,1,5,5
happy,0,28,3,4
relax,0,2,18,3
sad,0,1,11,12


In [55]:
from sklearn.metrics import classification_report
report = classification_report(x, y_pred)


In [57]:
print(report)

              precision    recall  f1-score   support

           0       0.49      0.78      0.60        23
           1       0.88      0.80      0.84        35
           2       0.50      0.50      0.50        24
           3       1.00      0.58      0.73        26

    accuracy                           0.68       108
   macro avg       0.72      0.66      0.67       108
weighted avg       0.74      0.68      0.69       108



In [58]:
accuracy = accuracy_score(x, y_pred) 

In [60]:
accuracy*100

67.5925925925926

In [22]:
from google.colab import files

# Prompt the user to upload a file
uploaded = files.upload()


Saving finalmusicfeatureextractednew.py to finalmusicfeatureextractednew.py


In [23]:
from finalmusicfeatureextractednew import feature_extraction

In [24]:
x = '/content/drive/MyDrive/TunePocket-Rock-Party-30-Sec-Intro-Preview (1).mp3'
features = {}
features = feature_extraction(x)

/content/finalmusicfeatureextractednew.py:131: RuntimeWarning: invalid value encountered in log2
  features['_HarmonicChangeDetectionFunction_PeriodEntropy'] = -np.sum(periodicity * np.log2(periodicity)) / np.sum(periodicity)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [25]:
attribute_avg = train['_HarmonicChangeDetectionFunction_PeriodEntropy'].mean()

In [28]:
features['_HarmonicChangeDetectionFunction_PeriodEntropy'] = attribute_avg

In [29]:
prediction2 = target_names[svm_classifier.predict(features)]
prediction2

array(['happy'], dtype=object)

In [30]:
y = '/content/drive/MyDrive/Happiness.mp3'
features_2 = {}
features_2 = feature_extraction(y)

/content/finalmusicfeatureextractednew.py:131: RuntimeWarning: invalid value encountered in log2
  features['_HarmonicChangeDetectionFunction_PeriodEntropy'] = -np.sum(periodicity * np.log2(periodicity)) / np.sum(periodicity)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [31]:
features_2['_HarmonicChangeDetectionFunction_PeriodEntropy'] = attribute_avg

In [32]:
prediction3 = target_names[svm_classifier.predict(features_2)]
prediction3

array(['happy'], dtype=object)

In [33]:
z = '/content/Energetic.mp3'
features_3 = {}
features_3 = feature_extraction(z)

/content/finalmusicfeatureextractednew.py:131: RuntimeWarning: invalid value encountered in log2
  features['_HarmonicChangeDetectionFunction_PeriodEntropy'] = -np.sum(periodicity * np.log2(periodicity)) / np.sum(periodicity)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [34]:
features_3['_HarmonicChangeDetectionFunction_PeriodEntropy'] = attribute_avg

In [35]:
prediction4 = target_names[svm_classifier.predict(features_3)]
prediction4

array(['happy'], dtype=object)

In [36]:
a = '/content/Sad Background Music.mp3'
features_4 = {}
features_4 = feature_extraction(a)

/content/finalmusicfeatureextractednew.py:131: RuntimeWarning: invalid value encountered in log2
  features['_HarmonicChangeDetectionFunction_PeriodEntropy'] = -np.sum(periodicity * np.log2(periodicity)) / np.sum(periodicity)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [37]:
features_4['_HarmonicChangeDetectionFunction_PeriodEntropy'] = attribute_avg

In [38]:
prediction5 = target_names[svm_classifier.predict(features_4)]
prediction5

array(['happy'], dtype=object)